<a href="https://colab.research.google.com/github/RomaTruba/dzSEM/blob/main/S_11ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание 1. Реализация семантического поиска с предобученными эмбеддингами

In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
# 1. Подготовка данных
documents = [
    "Коты обычно весят около 4-5 кг",
    "Собаки в среднем весят от 10 до 30 кг",
    "Автомобиль Tesla Model 3 имеет запас хода 500 км",
    "Яблоки содержат много витаминов и клетчатки"
]

In [3]:
# 2. Загрузка модели и создание эмбеддингов
model = SentenceTransformer('all-MiniLM-L6-v2')
document_embeddings = model.encode(documents)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
test_queries = [
    "Вес домашних животных",
    "Характеристики электромобилей",
    "Полезные свойства фруктов"
]

for query in test_queries:
    results = semantic_search(query, documents, document_embeddings, model)
    print(f"\nРезультаты для '{query}':")
    for doc, score in results:
        print(f"- {doc} (score: {score:.2f})")


Результаты для 'Вес домашних животных':
- Яблоки содержат много витаминов и клетчатки (score: 0.50)
- Собаки в среднем весят от 10 до 30 кг (score: 0.41)

Результаты для 'Характеристики электромобилей':
- Яблоки содержат много витаминов и клетчатки (score: 0.71)
- Собаки в среднем весят от 10 до 30 кг (score: 0.55)

Результаты для 'Полезные свойства фруктов':
- Яблоки содержат много витаминов и клетчатки (score: 0.73)
- Собаки в среднем весят от 10 до 30 кг (score: 0.57)


In [6]:
# 4. Функция семантического поиска
def semantic_search(query, documents, document_embeddings, model, top_k=2):
    query_embedding = model.encode([query])
    similarities = cosine_similarity(query_embedding, document_embeddings)[0]
    results = sorted(zip(documents, similarities), key=lambda x: x[1], reverse=True)[:top_k]
    return results

In [7]:

# 5. Пример использования с выводом результатов
query = "Какие животные самые тяжелые?"
search_results = semantic_search(query, documents, document_embeddings, model)

In [8]:
# 6. Вывод результатов
print(f"Результаты поиска для запроса: '{query}'\n")
for rank, (document, score) in enumerate(search_results, start=1):
    print(f"{rank}. {document} (релевантность: {score:.4f})")

Результаты поиска для запроса: 'Какие животные самые тяжелые?'

1. Яблоки содержат много витаминов и клетчатки (релевантность: 0.6510)
2. Коты обычно весят около 4-5 кг (релевантность: 0.5579)


In [9]:
# Альтернативный красивый вывод
from tabulate import tabulate

headers = ["Ранг", "Документ", "Релевантность"]
table = [[i, doc, f"{score:.4f}"] for i, (doc, score) in enumerate(results, 1)]
print(tabulate(table, headers=headers, tablefmt="grid"))

+--------+---------------------------------------------+-----------------+
|   Ранг | Документ                                    |   Релевантность |
+========+=============================================+=================+
|      1 | Яблоки содержат много витаминов и клетчатки |           0.637 |
+--------+---------------------------------------------+-----------------+
|      2 | Коты обычно весят около 4-5 кг              |           0.561 |
+--------+---------------------------------------------+-----------------+


Задание 2.  Анализ метрик сходства

In [13]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
import numpy as np

def compare_metrics(query_embedding, doc_embeddings, documents):
    # Косинусное сходство (чем ближе к 1, тем лучше)
    cos_sim = cosine_similarity(query_embedding, doc_embeddings)[0]

    # Евклидово расстояние (чем меньше, тем лучше)
    eucl_dist = euclidean_distances(query_embedding, doc_embeddings)[0]

    # Манхэттенское расстояние (чем меньше, тем лучше)
    manh_dist = manhattan_distances(query_embedding, doc_embeddings)[0]

    # Сортируем результаты по каждой метрике
    results = {
        "cosine": sorted(zip(documents, cos_sim), key=lambda x: x[1], reverse=True),
        "euclidean": sorted(zip(documents, eucl_dist), key=lambda x: x[1]),
        "manhattan": sorted(zip(documents, manh_dist), key=lambda x: x[1]),
    }
    return results

# Пример использования
query = "Тяжелые животные"
query_embedding = model.encode([query])
results = compare_metrics(query_embedding, document_embeddings, documents)

for metric, ranked_docs in results.items():
    print(f"\nМетрика: {metric}")
    for doc, score in ranked_docs[:2]:
        print(f"Документ: {doc} | Оценка: {score:.4f}")


Метрика: cosine
Документ: Яблоки содержат много витаминов и клетчатки | Оценка: 0.5652
Документ: Коты обычно весят около 4-5 кг | Оценка: 0.5173

Метрика: euclidean
Документ: Яблоки содержат много витаминов и клетчатки | Оценка: 0.9325
Документ: Коты обычно весят около 4-5 кг | Оценка: 0.9825

Метрика: manhattan
Документ: Яблоки содержат много витаминов и клетчатки | Оценка: 14.5698
Документ: Коты обычно весят около 4-5 кг | Оценка: 15.3657


Задание 3. Оптимизация и масштабирование (FAISS vs. Annoy)

In [16]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 65.4 MB/s eta 0:00:00


In [18]:
!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 28.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp311-cp311-linux_x86_64.whl size=551661 sha256=41bfe6472858593db9a0c6371246100516000c70e26111cfb6f0c5cc99c624f5
  Stored in directory: /root/.cache/pip/wheels/33/e5/58/0a3e34b92bedf09b4c57e37a63ff395ade6f6c1099ba59877c
Successfully built annoy


In [19]:
import time
import faiss
from annoy import AnnoyIndex

# FAISS
d = document_embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(d)
faiss_index.add(document_embeddings.astype('float32'))

# Annoy
annoy_index = AnnoyIndex(d, 'angular')  # 'angular' ≈ косинусное расстояние
for i, vec in enumerate(document_embeddings):
    annoy_index.add_item(i, vec)
annoy_index.build(10)  # 10 деревьев

# Тестируем скорость
query_embedding = model.encode(["Животные"]).astype('float32')[0]

# FAISS поиск
start = time.time()
D, I = faiss_index.search(query_embedding.reshape(1, -1), k=2)
faiss_time = time.time() - start

# Annoy поиск
start = time.time()
annoy_results = annoy_index.get_nns_by_vector(query_embedding, 2, include_distances=True)
annoy_time = time.time() - start

print(f"FAISS: {faiss_time:.5f} сек. | Annoy: {annoy_time:.5f} сек.")

FAISS: 0.00023 сек. | Annoy: 0.00020 сек.


Задание 4.Оценка качества (Precision, Recall, F1)

In [20]:
from sklearn.metrics import precision_recall_fscore_support

# Тестовые данные (запрос → релевантные документы)
test_data = {
    "Животные": [0, 1],  # Индексы документов про котов и собак
    "Техника": [2],       # Индекс документа про Tesla
    "Еда": [3]            # Индекс документа про яблоки
}

def evaluate_search(test_data, documents, model, top_k=2):
    metrics = []
    for query, relevant_indices in test_data.items():
        query_embedding = model.encode([query])
        # Ищем топ-2 документа (можно использовать FAISS/Annoy)
        D, I = faiss_index.search(query_embedding.astype('float32'), top_k)
        retrieved_indices = I[0].tolist()

        # Вычисляем метрики
        y_true = [1 if i in relevant_indices else 0 for i in range(len(documents))]
        y_pred = [1 if i in retrieved_indices else 0 for i in range(len(documents))]

        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
        metrics.append((query, precision, recall, f1))

    return metrics

results = evaluate_search(test_data, documents, model)
for query, prec, rec, f1 in results:
    print(f"Запрос: '{query}' | Precision: {prec:.2f}, Recall: {rec:.2f}, F1: {f1:.2f}")

Запрос: 'Животные' | Precision: 0.50, Recall: 0.50, F1: 0.50
Запрос: 'Техника' | Precision: 0.00, Recall: 0.00, F1: 0.00
Запрос: 'Еда' | Precision: 0.50, Recall: 1.00, F1: 0.67


Задание 5.Веб-приложение на Streamlit

In [22]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 124.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.5 MB/s eta 0:00:00


In [29]:
import streamlit as st
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Загрузка модели и индекса
model = SentenceTransformer('all-MiniLM-L6-v2')
documents = ["Коты...", "Собаки...", "Tesla...", "Яблоки..."]
embeddings = model.encode(documents)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings.astype('float32'))

st.title("Семантический поиск")
query = st.text_input("Введите запрос:")

if query:
    query_embedding = model.encode([query]).astype('float32')
    D, I = index.search(query_embedding, 2)
    for i, idx in enumerate(I[0]):
        st.write(f"{i+1}. {documents[idx]} (расстояние: {D[0][i]:.2f})")
    st.button(":))")


2025-04-06 16:54:40.406 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 16:54:40.406 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 16:54:40.408 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 16:54:40.409 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 16:54:40.411 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 16:54:40.411 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 16:54:40.412 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 16:54:40.414 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
streamlit run app.py